In [101]:
import os
from PIL import Image
import cv2

# Validate images using OpenCV
def validate_images_with_opencv(directory):
    invalid_images = []
    for subdir, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.jpg'):
                file_path = os.path.join(subdir, file)
                img = cv2.imread(file_path)
                if img is None:
                    invalid_images.append(file_path)
                    print(f"Invalid image file: {file_path}")
                    os.remove(file_path)  # Remove invalid image
    return invalid_images

# Validate all directories (train, valid, and test)
train_dir = 'yoga_combined/train'
valid_dir = 'yoga_combined/valid'
test_dir = 'yoga_combined/test'

# Run validation on each directory
train_invalid_images = validate_images_with_opencv(train_dir)
valid_invalid_images = validate_images_with_opencv(valid_dir)
test_invalid_images = validate_images_with_opencv(test_dir)


In [102]:
import tensorflow as tf
import os

def load_image(file_path):
    try:
        # Load the image, decode and resize
        image = tf.io.read_file(file_path)
        image = tf.image.decode_jpeg(image, channels=3)  # Decode as RGB
        image = tf.image.resize(image, [224, 224])  # Resize to target size
        image = image / 255.0  # Normalize pixel values to [0, 1]
        return image
    except tf.errors.InvalidArgumentError:
        return None  # Return None for invalid images

def prepare_data(input_dir, batch_size=32):
    images = []
    labels = []
    
    # Paths to train, validation, and test directories
    for split in ['train', 'valid', 'test']:
        split_dir = os.path.join(input_dir, split)
        for label, class_name in enumerate(os.listdir(split_dir)):
            class_dir = os.path.join(split_dir, class_name)
            if os.path.isdir(class_dir):
                for file_name in os.listdir(class_dir):
                    file_path = os.path.join(class_dir, file_name)
                    image = load_image(file_path)
                    if image is not None:
                        images.append(image)
                        labels.append(label)
    
    # Convert to TensorFlow Datasets
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    
    # Shuffle, repeat, and batch
    dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

# Prepare the datasets
input_dir = 'yoga_combined'
train_ds = prepare_data(input_dir)
valid_ds = prepare_data(input_dir)


In [103]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

def build_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze the base model

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dense(6, activation='softmax')  # Output layer for 6 classes
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Build the model
model = build_model()


In [104]:
def train_model(model, train_ds, valid_ds, epochs=10):
    history = model.fit(
        train_ds,
        validation_data=valid_ds,
        epochs=epochs
    )
    return history

# Train the model
history = train_model(model, train_ds, valid_ds)


Epoch 1/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 166s 2s/step - accuracy: 0.6073 - loss: 1.2083 - val_accuracy: 0.9224 - val_loss: 0.2571
Epoch 2/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - accuracy: 0.9366 - loss: 0.2076 - val_accuracy: 0.9815 - val_loss: 0.0874
Epoch 3/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.9790 - loss: 0.0859 - val_accuracy: 0.9897 - val_loss: 0.0674
Epoch 4/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.9869 - loss: 0.0593 - val_accuracy: 0.9931 - val_loss: 0.0415
Epoch 5/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 139s 2s/step - accuracy: 0.9891 - loss: 0.0408 - val_accuracy: 0.9996 - val_loss: 0.0187
Epoch 6/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - accuracy: 0.9997 - loss: 0.0167 - val_accuracy: 1.0000 - val_loss: 0.0129
Epoch 7/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 123s 2s/step - accuracy: 1.0000 - loss: 0.0146 - val_accuracy: 1.0000 - val_loss: 0.0099
Epoch 8/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 1.0000 - loss: 0.0115 - val_accuracy: 1.0000 - v

In [105]:
def evaluate_model(model, valid_ds):
    results = model.evaluate(valid_ds)
    print(f"Validation loss: {results[0]}")
    print(f"Validation accuracy: {results[1]}")

# Evaluate the model on the validation dataset
evaluate_model(model, valid_ds)


73/73 ━━━━━━━━━━━━━━━━━━━━ 70s 954ms/step - accuracy: 1.0000 - loss: 0.0045
Validation loss: 0.004691411275416613
Validation accuracy: 1.0


In [109]:
def prepare_data(input_dir, batch_size=32, img_size=(224, 224)):
    # Define paths for training, validation, and test data
    train_dir = os.path.join(input_dir, 'train')
    valid_dir = os.path.join(input_dir, 'valid')
    test_dir = os.path.join(input_dir, 'test')  # Assuming you have a separate test folder
    
    # Load data using ImageDataGenerator
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

    # Prepare datasets
    train_ds = train_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='sparse'
    )
    
    valid_ds = valid_datagen.flow_from_directory(
        valid_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='sparse'
    )
    
    test_ds = test_datagen.flow_from_directory(
        test_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='sparse'
    )
    
    return train_ds, valid_ds, test_ds


In [110]:
train_ds, valid_ds, test_ds = prepare_data(input_dir)


Found 1718 images belonging to 6 classes.
Found 300 images belonging to 6 classes.
Found 302 images belonging to 6 classes.


In [111]:
def evaluate_model_on_test(model, test_ds):
    results = model.evaluate(test_ds)
    print(f"Test loss: {results[0]}")
    print(f"Test accuracy: {results[1]}")

# Assuming you have a test dataset ready, you can call:
evaluate_model_on_test(model, test_ds)



c:\Users\HP\anaconda3\envs\pose\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 5/10 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 1.0000 - loss: 0.0038

c:\Users\HP\anaconda3\envs\pose\lib\site-packages\PIL\Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 1.0000 - loss: 0.0058
Test loss: 0.008419248275458813
Test accuracy: 1.0


In [112]:
# Assuming you have a test dataset prepared
evaluate_model_on_test(model, test_ds)


10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 1.0000 - loss: 0.0065
Test loss: 0.008419249206781387
Test accuracy: 1.0


In [113]:
# Save the trained model
model.save('yoga_pose_detection_model.h5')


In [114]:
model.save('yoga_pose_detection_model.h5')


In [115]:
model.save('yoga_pose_detection_model.keras')
